# UI helpers & utilities

> Utility functions and decorators for common SSE patterns in FastHTML. Includes the @sse_element 
  decorator and helper functions for rapid SSE integration.

In [ ]:
#| default_exp helpers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Any, Callable, List, Optional, Union

from fasthtml.common import FT, Script

In [ ]:
#| export
# Helper function for creating OOB swap elements
def oob_swap(
    element: Any,    # The element to add OOB swap to
    swap_type: str = "outerHTML",  # Type of swap (innerHTML, outerHTML, beforebegin, afterend, etc.)
    target_id: Optional[str] = None  # Optional target ID (uses element's ID if not specified)
):
    """Add OOB swap attributes to an element."""
    if hasattr(element, 'attrs'):
        element.attrs['hx-swap-oob'] = swap_type
        if target_id:
            element.attrs['hx-swap-oob'] = f"{swap_type}:#{target_id}"
    return element

In [ ]:
#| export
def oob_element(
    element_id: str,  # ID of the target element
    content: Any,    # Content to swap
    swap_type: str = "innerHTML"  # Type of swap
):
    """Create a wrapper element for OOB swap."""
    return Div(
        content,
        id=element_id,
        hx_swap_oob=swap_type
    )

In [ ]:
#| export
def sse_element(endpoint: str, 
                events: Optional[Union[str, List[str]]] = None, # Event name(s) to listen for from SSE stream
                auto_close: bool = True,  # Whether to auto-close on completion
                swap_type: str = "message" # How to swap content
               ):
    """Decorator to add SSE capabilities to any element."""
    def decorator(
        element_func: Callable  # Function that creates an element
    ):
        """Inner decorator function."""
        def wrapper(*args, **kwargs):
            """Wrap element creation with SSE attributes."""
            # Create the base element
            element = element_func(*args, **kwargs)
            
            # Add SSE attributes
            return htmx_sse.add_sse_attrs(
                element,
                endpoint=endpoint,
                events=events,
                swap_type=swap_type,
                auto_reconnect=not auto_close
            )
        return wrapper
    return decorator

In [ ]:
#| export
# Simplified OOB update function (already exists but making it more prominent)
def oob_update(
    element_id: str,  # Target element ID
    content: Any,  # Content to swap
    swap_type: str = "innerHTML"  # Type of swap (innerHTML, outerHTML, etc.)
):
    """Create an out-of-band update element."""
    return oob_element(element_id, content, swap_type)

In [ ]:
#| export
def cleanup_sse_on_unload(
) -> FT:  # FastHTML element (Script) for cleanup
    """Add script to cleanup SSE connections on page unload."""
    return Script("""window.addEventListener('beforeunload',()=>{document.querySelectorAll('[sse-connect]').forEach(e=>{e['htmx-internal-data']?.sseEventSource?.close()});htmx?.findAll('[sse-connect]').forEach(e=>htmx.trigger(e,'htmx:sseClose'))});""")

In [ ]:
#| export
# 
def get_htmx_idx(
    hdrs: List  # List of header elements to search
) -> int:  # Index of HTMX script or -1 if not found
    """Find the index of HTMX script in headers list."""
    return next((i for i, hdr in enumerate(hdrs) if (hdr.attrs.get('src') or '').endswith('htmx.min.js')), -1)

In [ ]:
#| export
def insert_htmx_sse_ext(
    hdrs: List  # List of header elements to modify
):
    """Add HTMX SSE extension after HTMX script"""
    htmx_idx = get_htmx_idx(hdrs)
    if htmx_idx >= 0:
        # Insert SSE extension right after HTMX
        hdrs.insert(htmx_idx+1, Script(src="https://unpkg.com/htmx-ext-sse"))
        hdrs.insert(htmx_idx+2, cleanup_sse_on_unload())
    else:
        print("HTMX not found")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()